In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Building the network

In [3]:
def dhdt(h, I, tau, R):
    return (R * I - h) / tau
def activation(h, r0, alpha, beta):
    return r0 * (1/(1 + np.exp(-2 * alpha * (h - beta))))

In [ ]:
T = 1 # s
dt = 1e-2 # s which is 1 ms
tau = 1e-2 # s
R = 1e6 # Ohm
stimulus = np.zeros(int(T/dt))
N = 100 # number of neurons
h = np.zeros(N) # mV
r0 = 1000 # Hz
alpha = 2 # mV^-1